# Running the `ialg` algorithm on the TSPLIB and the HardTSPLIB instances

Here, we want to compute the minimum number of SECs needed to prove optimality for some small famous instances in the TSPLIB [1] and some hard-to-solve instances in the HardTSPLIB [2]. We will use the `ialg` algorithm to compute the minimum number of SECs needed to prove optimality for these instances.

### References
[1] Reinelt, Gerhard. "TSPLIB—A traveling salesman problem library." ORSA journal on computing 3.4 (1991): 376-384.

[2] Vercesi, Eleonora, et al. "On the generation of metric TSP instances with a large integrality gap by branch-and-cut." Mathematical Programming Computation 15.2 (2023): 389-416.


In [1]:
# Suppress future warning of pandas
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Things that you actually need
from ialg import ialg, mip
from cover import set_cover_subroutine
from utils import from_tsplib_file_to_graph
import pandas as pd

## Download the data
You can download the files of the TSPLIB from [here](http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/) and the HardTSPLIB from [here](https://github.com/eleonoravercesi/HardTSPLIB/). Then, you can put the files in the `data` folder.

## TSPLIB instances

These are the instances of the TSPLIB with a number of nodes that is $\leq 100$. Note that in this notebook we run experiments only when $n \leq 42$. You can run more experiments, but it will take more time.

In [2]:
tsplib_instances = [("burma14", 14), ("ulysses16", 16), ("gr17", 17), ("gr21", 21), ("ulysses22", 22),  ("gr24", 24),
                    ("fri26", 29), ("bayg29", 29), ("bays29", 29), ("dantzig42", 42), ("swiss42", 42)]
# In case you want to test also these instances
large_tsplib_instance = [("att48", 48), ("gr48", 48), ("hk48", 48), ("eil51", 51), ("berlin52", 52), ("brazil58", 58), ("st70", 70),
                     ("eil76", 76), ("pr76", 76), ("gr96", 96), ("rat99", 99), ("kroA100", 100), ("kroB100", 100),
                    ("kroC100", 100), ("kroD100", 100), ("kroE100", 100), ("rd100", 100)]

Now, we run the `ialg` algorithm on the TSPLIB instances. Here, we set a flag `minim`, that you can either set equal to True or False, to run also the minimalization procedure

In [3]:
minim = True

In [4]:
# Create a dataframe with keys tsplib instances
df = pd.DataFrame(index = [x[0] for x in tsplib_instances], columns=["S_min", "b_prime", "runtime", "BeB_n_nodes"])
for instance_name, n in tsplib_instances:
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    (S_family, size_S_family, partitions, c, runtime, num_nodes) = ialg(G, verbose=True, minimalize=minim)
    df.loc[instance_name] = [size_S_family, c, runtime, num_nodes]

    if size_S_family == -1:
        print("Ran into time limit.")
        continue

    # check that k* >= size_S_family
    partitions_list = [ [ list(part) for part in partition ] for npts in partitions for partition in partitions[npts] ]
    smallest_S_family = set_cover_subroutine(partitions_list, verbose=True)
    print("k* =",size_S_family,"for S_family =", smallest_S_family)
    assert len(smallest_S_family) == size_S_family

    # Compute the TSP
    tsp_cost = mip(G, subtour_callbacks=True, verbose=False)

    # check that k* <= size_S_family
    two_factor_cost_with_S_family, _ = mip(G, initial_subtours=S_family, verbose=False, return_edges=True)
    assert round(two_factor_cost_with_S_family) == round(tsp_cost)

******* Instance: burma14 *******
Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-28
TSP compute in 0.03851127624511719 seconds. TSP cost = 3323
With smart initialization, we begin with #SECs = 2
They are:
[2, 3, 4, 5, 6, 11, 12, 13]
[0, 1, 2, 3, 4, 5, 6, 7, 11, 12, 13]
Found a solution with #SECs: 2
Specifically, they are:
[2, 3, 4, 5, 6, 11, 12, 13]
[0, 1, 2, 3, 4, 5, 6, 7, 11, 12, 13]
Set parameter MIPGap to value 1e-16
Set parameter FeasibilityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 4 columns and 4 nonzeros
Model fingerprint: 0xbbc18e87
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS

In [5]:
df

,S_min,b_prime,runtime,BeB_n_nodes
burma14,2,2,0.005698,1
ulysses16,4,2,0.00622,1
gr17,5,2,0.155492,5
gr21,0,0,0.00935,1
ulysses22,5,2,0.019617,2
gr24,1,2,0.079889,2
fri26,4,4,1.83468,6
bayg29,4,2,0.069777,4
bays29,5,3,0.18854,5
dantzig42,4,2,0.113816,3


## HardTSPLIB instances

HardTSPLIB is made of instances generated both at random and starting from instances of the TSPLIB.  We divide such instances into to, to make direct comparison with TSPLIB. `hardtsplib_instances_random` and `hardtsplib_instances_tsplib`. We will only run the algorithm on the instances that are feasible to run on our machine. 

In [6]:
hardtsplib_instances_random = [("10001_hard", 10), ("10007_hard", 10), ("10008_hard", 10), ("10010_hard", 10), ("11675_hard", 11), ("12290_hard", 12), ("14850_hard", 14), ("15002_hard", 15), ("15005_hard", 15), ("15007_hard", 15), ("16038_hard", 16), ("20004_hard", 20), ("20007_hard", 20)]

hardtsplib_instances_random_large = [ ("20009_hard", 20), ("20181_hard", 20), ("25001_hard", 25), ("25004_hard", 25), ("25006_hard", 25), ("30001_hard", 30), ("30003_hard", 30), ("30005_hard", 30), ("33001_hard", 33), ("35002_hard", 35), ("35003_hard", 35), ("35009_hard", 35), ("40003_hard", 40), ("40004_hard", 40), ("40008_hard", 40)]

hardtsplib_instances_tsplib = [("gr24_hard", 24), ("bayg29_hard", 29), ("bays29_hard", 29)]

hardtsplib_instances_tsplib_large = [("dantzig42_hard", 42),  ("gr48_hard", 48), ("hk48_hard", 48), ("att48_hard", 48), ("eil51_hard", 51), ("brazil58_hard", 58), ("st70_hard", 70), ("pr76_hard", 76)]


## HardTSPLIB instances derived from TSPLIB

Note: in this case even the smallest instances require more time.

In [7]:
minim = True

In [8]:
# Create a dataframe with keys tsplib instances
df = pd.DataFrame(index = [x[0] for x in hardtsplib_instances_tsplib], columns=["S_min", "b_prime", "runtime", "BeB_n_nodes"])
for instance_name, n in hardtsplib_instances_tsplib:
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    (S_family, size_S_family, partitions, c, runtime, num_nodes) = ialg(G, verbose=True, minimalize=minim, smart_initialize=True)
    df.loc[instance_name] = [size_S_family, c, runtime, num_nodes]

    if size_S_family == -1:
        print("Ran into time limit.")
        continue

    # check that k* >= size_S_family
    partitions_list = [ [ list(part) for part in partition ] for npts in partitions for partition in partitions[npts] ]
    smallest_S_family = set_cover_subroutine(partitions_list, verbose=True)
    print("k* =",size_S_family,"for S_family =", smallest_S_family)
    assert len(smallest_S_family) == size_S_family

    # Compute the TSP
    tsp_cost = mip(G, subtour_callbacks=True, verbose=False)

    # check that k* <= size_S_family
    two_factor_cost_with_S_family, _ = mip(G, initial_subtours=S_family, verbose=False, return_edges=True)
    assert round(two_factor_cost_with_S_family) == round(tsp_cost)

******* Instance: gr24_hard *******
TSP compute in 0.33020639419555664 seconds. TSP cost = 1000
With smart initialization, we begin with #SECs = 6
They are:
[0, 1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22]
[4, 5, 6, 7, 20, 9, 23]
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 22, 23]
[4, 5, 6, 7, 20, 23]
[17, 2, 10, 21]
[4, 5, 6, 7, 9, 16, 20, 23]
Found smaller partition: 5 -> 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 6 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 2 7 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 3 8 2
Found smaller partition: 4 -> 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 4 9 2
Found smaller partition: 4 -> 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 5 10 2
Found smaller partition: 4 -> 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 6 11 2
Found smaller partition: 4 -> 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 7 12 2
Found smaller partition: 4 -> 2
num_bb_nodes, 

In [9]:
df

,S_min,b_prime,runtime,BeB_n_nodes
gr24_hard,63,2,10.539447,58
bayg29_hard,134,2,313.553131,133
bays29_hard,125,2,113.64403,113


### Comparison of the `ialg` algorithm on the TSPLIB and HardTSPLIB instances
In this case, we can see a line-by-line comparison between the values of TSPLIB and HardTSPLIB
    

In [10]:
df = pd.DataFrame(index = [x[0].split("_")[0] for x in hardtsplib_instances_tsplib], columns=["S_min_TSPLIB", "S_min_HardTSPLIB", "b_prime_TSPLIB", "b_prime_HardTSPLIB", "runtime_TSPLIB", "runtime_HardTSPLIB", "BeB_n_nodes_TSPLIB", "BeB_n_nodes_HardTSPLIB"])
for instance_name, n in hardtsplib_instances_tsplib:
    instance_name_short = instance_name.split("_")[0]
    G_TSPLIB = from_tsplib_file_to_graph("./data/" + instance_name_short + ".tsp")
    (S_family_TSPLIB, size_S_family_TSPLIB, partitions_TSPLIB, b_TSPLIB, runtime_TSPLIB, num_nodes_TSPLIB) = ialg(G_TSPLIB, verbose=False, minimalize=minim)
    print("Instance:", instance_name_short)
    print("Time TSPLIB:", runtime_TSPLIB)
    G_HardTSPLIB = from_tsplib_file_to_graph("./data/" + instance_name + ".tsp")
    (S_family_HardTSPLIB, size_S_family_HardTSPLIB, partitions_HardTSPLIB, b_HardTSPLIB, runtime_HardTSPLIB, num_nodes_HardTSPLIB) = ialg(G_HardTSPLIB, verbose=False, minimalize=minim)
    print("Time HardTSPLIB:", runtime_HardTSPLIB)
    df.loc[instance_name_short] = [size_S_family_TSPLIB, size_S_family_HardTSPLIB, b_TSPLIB, b_HardTSPLIB, runtime_TSPLIB, runtime_HardTSPLIB, num_nodes_TSPLIB, num_nodes_HardTSPLIB]

Instance: gr24
Time TSPLIB: 0.11471741698915139
Time HardTSPLIB: 16.19633958299528
Instance: bayg29
Time TSPLIB: 0.08733641699654981
Time HardTSPLIB: 302.53696412499994
Instance: bays29
Time TSPLIB: 0.20998262500506826
Time HardTSPLIB: 104.89335879198916


In [11]:
df

,S_min_TSPLIB,S_min_HardTSPLIB,b_prime_TSPLIB,b_prime_HardTSPLIB,runtime_TSPLIB,runtime_HardTSPLIB,BeB_n_nodes_TSPLIB,BeB_n_nodes_HardTSPLIB
gr24,1,63,2,2,0.114717,16.19634,2,58
bayg29,4,134,2,2,0.087336,302.536964,4,133
bays29,5,125,3,2,0.209983,104.893359,5,113



## Hard instances derived from random instances

In [12]:
minim = True

In [13]:
# Create a dataframe with keys tsplib instances
df = pd.DataFrame(index = [x[0] for x in hardtsplib_instances_random], columns=["S_min", "b_prime", "runtime", "BeB_n_nodes"])
for instance_name, n in hardtsplib_instances_random:
    # Parse the instance
    G = from_tsplib_file_to_graph("./data/" + instance_name)
    print("******* Instance:", instance_name, "*******")
    (S_family, size_S_family, partitions, c, runtime, num_nodes) = ialg(G, verbose=True, minimalize=minim, smart_initialize=True)
    df.loc[instance_name] = [size_S_family, c, runtime, num_nodes]

    if size_S_family == -1:
        print("Ran into time limit.")
        continue

    # check that k* >= size_S_family
    partitions_list = [ [ list(part) for part in partition ] for npts in partitions for partition in partitions[npts] ]
    smallest_S_family = set_cover_subroutine(partitions_list, verbose=True)
    print("k* =",size_S_family,"for S_family =", smallest_S_family)
    assert len(smallest_S_family) == size_S_family

    # Compute the TSP
    tsp_cost = mip(G, subtour_callbacks=True, verbose=False)

    # check that k* <= size_S_family
    two_factor_cost_with_S_family, _ = mip(G, initial_subtours=S_family, verbose=False, return_edges=True)
    assert round(two_factor_cost_with_S_family) == round(tsp_cost)

******* Instance: 10001_hard *******
TSP compute in 0.024255037307739258 seconds. TSP cost = 1000
With smart initialization, we begin with #SECs = 2
They are:
[2, 3, 4, 5, 6, 7]
[0, 1, 9]
num_bb_nodes, num_subtour_constrs, num_conn_comp = 1 2 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 2 3 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 3 4 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 4 5 2
num_bb_nodes, num_subtour_constrs, num_conn_comp = 5 6 2
Found a solution with #SECs: 7
Specifically, they are:
[2, 3, 4, 5, 6, 7]
[0, 1, 9]
[2, 6, 7]
[8, 1, 9]
[0, 1, 5, 8, 9]
[0, 9, 5]
[0, 9, 5, 1]
Set parameter MIPGap to value 1e-16
Set parameter FeasibilityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 14 columns and 14 nonzeros
Model fingerprint: 0xd1a8

In [14]:
df

,S_min,b_prime,runtime,BeB_n_nodes
10001_hard,7,2,0.030176,6
10007_hard,6,2,0.016797,5
10008_hard,7,2,0.016919,5
10010_hard,7,2,0.023377,5
11675_hard,8,2,0.058563,8
12290_hard,13,2,0.148521,12
14850_hard,19,2,0.525227,19
15002_hard,22,2,0.584678,18
15005_hard,21,2,0.643508,18
15007_hard,27,2,1.115755,22
